In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-24"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13391,2024-02-24,Eua Ncaa,15:00,Stony Brook,William & Mary,1.25,4.19,138.5,1.80,1.95,0.0,0.0,0.0,KbYvhGTJ,0.800000,0.238663,0.555556,0.512821,0.038663,106.618,16.052267,0.150559,2.4,1.341641,0.559017,85.60,1.400,0.285132,0.203665,37.0,409.390,292.069218,0.713425,0.0,0.000000,NaN,319.15,6.974,5.198310,0.745384,-48.0,80,65,1.07,4.91,125.054,473.960,0.764299,0.056569,NaN,3.06,0.612,0.408497,0.586186,0.8,0.213814,-1.28,-0.256,-12.460938,0.263649,0.2,-0.063649
13392,2024-02-24,Eua Ncaa,16:00,Ball State,Eastern Michigan,1.20,4.80,143.5,1.91,1.91,-10.5,2.0,11.0,nTEyCIWO,0.833333,0.208333,0.523560,0.523560,0.041667,139.740,15.368734,0.109981,1.2,1.643168,1.369306,134.40,2.032,0.087864,0.043240,-17.0,238.128,90.244546,0.378975,0.0,0.000000,NaN,143.52,3.476,0.982258,0.282583,-60.0,70,69,1.92,2.08,164.198,243.492,0.848528,0.000000,0.979071,-3.49,-0.698,-0.286533,0.596028,0.5,-0.096028,-5.00,-1.000,-3.800000,0.305532,0.1,-0.205532
13393,2024-02-24,Eua Ncaa,15:00,VMI,Western Carolina,11.50,1.05,154.5,1.91,1.91,0.0,0.0,0.0,hncBpGMD,0.086957,0.952381,0.523560,0.523560,0.039337,658.420,212.703178,0.323051,0.0,0.000000,NaN,326.40,9.990,3.100887,0.310399,-102.0,126.360,49.997608,0.395676,1.8,1.643168,0.912871,93.10,1.760,0.857817,0.487396,17.0,51,70,6.40,1.33,568.342,153.540,1.177572,0.000000,NaN,-0.62,-0.124,-84.677419,0.000000,0.0,0.000000,-1.50,-0.300,-0.166667,0.511464,0.5,-0.011464
13394,2024-02-24,Eua Ncaa,15:00,UMass Lowell,Albany,1.21,4.67,161.5,1.80,1.95,0.0,0.0,0.0,OO2nZe4I,0.826446,0.214133,0.555556,0.512821,0.040579,98.652,15.865317,0.160821,1.8,1.643168,0.912871,105.27,1.254,0.047223,0.037658,11.0,173.356,92.165578,0.531655,0.6,1.341641,2.236068,335.20,2.230,1.159634,0.520015,-18.0,87,80,1.21,4.19,96.270,194.462,0.832173,0.056569,NaN,-1.45,-0.290,-0.724138,0.000000,0.0,0.000000,-2.53,-0.506,-7.252964,0.425955,0.3,-0.125955
13395,2024-02-24,Eua Ncaa,15:00,New Hampshire,UMBC Retrievers,1.43,2.92,163.5,1.80,1.95,0.0,0.0,0.0,OEs7WTp0,0.699301,0.342466,0.555556,0.512821,0.041766,204.664,135.661413,0.662849,1.8,1.643168,0.912871,97.94,2.882,1.554580,0.539410,-11.0,228.398,91.715351,0.401559,1.2,1.643168,1.369306,131.60,3.060,1.228983,0.401629,-5.0,83,56,1.18,2.35,110.880,460.500,0.484409,0.056569,NaN,-0.87,-0.174,-2.471264,0.651821,0.7,0.048179,0.60,0.120,16.000000,0.367515,0.3,-0.067515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13650,2024-02-24,Itália Série A1 Feminina,14:30,Sassari F,Ragusa F,2.53,1.48,142.5,1.80,1.86,0.0,0.0,0.0,dEM7q7N2,0.395257,0.675676,0.555556,0.537634,0.070933,224.234,183.803005,0.819693,0.6,1.341641,2.236068,551.80,3.558,3.012701,0.846740,-53.0,140.792,79.027970,0.561310,1.8,1.643168,0.912871,98.28,2.098,1.471146,0.701213,41.0,62,63,8.90,1.56,0.000,0.000,0.370305,0.023184,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13651,2024-02-24,Japão B2.League,02:00,Kobe,Nara,1.51,2.45,149.5,1.84,1.86,0.0,0.0,0.0,SWJVd9lH,0.662252,0.408163,0.543478,0.537634,0.070415,130.692,68.571387,0.524679,0.6,1.341641,2.236068,78.40,1.704,0.724555,0.425208,-15.0,149.956,38.091963,0.254021,1.

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
13405,16:00,Eua Ncaa,North Dakota St.,North Dakota,1.77,Back Home
13406,16:00,Eua Ncaa,Le Moyne,Fairleigh Dickinson,1.69,Back Home
13413,16:00,Eua Ncaa,South Dakota Coyotes,Oral Roberts,2.18,Back Home
13414,16:00,Eua Ncaa,Stetson,Kennesaw State,1.76,Back Home
13439,08:45,Turquia Tbl,Final Spor,Bornova,1.89,Back Home
13469,22:00,Eua Ncaa,Idaho State,Weber State,2.18,Back Home
13492,18:00,Eua Ncaa,Bellarmine,North Alabama,2.28,Back Home
13505,18:00,Eua Ncaa,Utah Valley,Stephen F. Austin,1.58,Back Home
13528,14:00,Eua Ncaa,Towson,Coll Of Charltn,1.90,Back Home
13576,13:30,Bósnia E Herzegovina Prvenstvo Bih Feminina,Play Off Sarajevo F,Zrinjski F,1.63,Back Home
